In [18]:
import os
import gc
import sys
import glob
import copy
import numpy as np
import pandas as pd
import netCDF4 as nc
import multiprocessing as mp
from datetime import datetime, timedelta
from matplotlib.cm import get_cmap
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib import colors
import cartopy.crs as ccrs

In [19]:
# To use PLUMBER2_GPP_common_utils, change directory to where it exists
os.chdir('/g/data/w97/mm3972/scripts/Land_Drought_Rainfall')
from common_utils import *

### Compare PLUMBER2 site

In [ ]:
def get_PLUMBER2_lat_lon_LAI(PLUMBER2_AU_file, au_site_list, start_date, end_date):

    site_name  = PLUMBER2_AU_file.split('_')[0] 
    print(site_name)

    # Read PLUMBER2 file
    f_PLUMBER2 = nc.Dataset(f'{PLUMBER2_path}/{PLUMBER2_AU_file}') 

    df_P2      = pd.DataFrame(f_PLUMBER2.variables['LAI'][:,0,0].data, columns=['LAI'])

    # Read PLUMBER2 time
    P2_time    = nc.num2date(f_PLUMBER2.variables['time'][:], 
                             f_PLUMBER2.variables['time'].units, 
                             only_use_cftime_datetimes=False,
                             only_use_python_datetimes=True)
        
    ntime = len(P2_time)
    year  = np.zeros(ntime)
    month = np.zeros(ntime)
    day   = np.zeros(ntime)
    
    # Extract year, month, day
    for tt, t in enumerate(P2_time):
        year[tt]  = t.year
        month[tt] = t.month
        day[tt]   = t.day

    # Make time array for processed LAI  
    start_date_P2  = datetime(int(year[0]),  int(month[0]),  int(day[0]), 0, 0)
    end_date_P2    = datetime(int(year[-1]), int(month[-1]), int(day[-1]), 0, 0)

    # Create an array of datetime objects
    P2_time_new    = np.array([start_date_P2 + timedelta(days=i) for i in range((end_date_P2 - start_date_P2).days + 1)])
    
    time_mask      = (P2_time_new>=start_date) & (P2_time_new<=end_date)
    
    if np.sum(time_mask)>10:

        df_P2['year']  = year
        df_P2['month'] = month
        df_P2['day']   = day
        df_P2          = df_P2.groupby(['year', 'month', 'day']).mean(numeric_only=True).reset_index()

        # Get lat and lon for the site
        lat = au_site_characters.loc[au_site_characters['site_name'] == site_name, 'lat'].values[0]
        lon = au_site_characters.loc[au_site_characters['site_name'] == site_name, 'lon'].values[0]
        # print(lat, lon)

        df_P2_tmp         = df_P2.loc[time_mask]
        df_P2_tmp['time'] = P2_time_new[time_mask]

        return site_name, lat, lon, df_P2_tmp
    else:
        return site_name, np.nan, np.nan, np.nan

In [ ]:
for year in np.arange(2000,2020):
    file_LAI_yuan      = f'/g/data/w97/mm3972/data/MODIS/MODIS_LAI/Sun_Yatsen_Uni_processed_MODIS_LAI_61/lai_8-day_0.05_{year}.nc'
    # file_LAI_processed = f'/g/data/w97/mm3972/scripts/Land_Drought_Rainfall/process_for_CABLE/nc_files/gridinfo_AWAP_OpenLandMap_ELEV_DLCM_fix_MODIS_LAI_albedo_lc_time_varying_{year}.nc'
    file_LAI_processed = f'/g/data/w97/mm3972/data/MODIS/MODIS_LAI/AUS/regrid_2_AWAP_5km_daily/remove_high_frequent_varibility_method2_smooth_anomaly/MCD15A3H.061_500m_aid0001_LAI_regridded_daily_2000-2023_remove_high_freq.nc'

    f_LAI_yuan         = nc.Dataset(file_LAI_yuan)
    f_LAI_processed    = nc.Dataset(file_LAI_processed)

    LAI_yuan           = f_LAI_yuan.variables['lai'][:]
    lat_yuan           = f_LAI_yuan.variables['lat'][:]
    lon_yuan           = f_LAI_yuan.variables['lon'][:]
    time_yuan          = nc.num2date(f_LAI_yuan.variables['time'][:],f'days since {year}-01-01 00:00:00',
                         only_use_cftime_datetimes=False, only_use_python_datetimes=True)

    LAI_processed      = f_LAI_processed.variables['LAI'][:]
    lat_processed      = f_LAI_processed.variables['latitude'][:]
    lon_processed      = f_LAI_processed.variables['longitude'][:]

    # Make time array for processed LAI  
    start_date         = datetime(year, 1, 1, 0, 0)
    end_date           = datetime(year, 12, 31, 0, 0)

    # Create an array of datetime objects
    # time_processed     = np.array([start_date + timedelta(days=i) for i in range((end_date - start_date).days + 1)])
    time_processed     = nc.num2date(f_LAI_processed.variables['time'][:],f_LAI_processed.variables['time'].units,
                         only_use_cftime_datetimes=False, only_use_python_datetimes=True)

    message            = 'Met'
    PLUMBER2_path      = '/g/data/w97/mm3972/data/Fluxnet_data/Post-processed_PLUMBER2_outputs/Nc_files/Met'
    PLUMBER2_AU_files  = [  f'AU-ASM_2011-2017_OzFlux_{message}.nc',      
                            f'AU-Cpr_2011-2017_OzFlux_{message}.nc',     
                            f'AU-Cum_2013-2018_OzFlux_{message}.nc',       
                            f'AU-DaP_2009-2012_OzFlux_{message}.nc',       
                            f'AU-DaS_2010-2017_OzFlux_{message}.nc',       
                            f'AU-Dry_2011-2015_OzFlux_{message}.nc',       
                            f'AU-Emr_2012-2013_OzFlux_{message}.nc',       
                            f'AU-GWW_2013-2017_OzFlux_{message}.nc',       
                            f'AU-Gin_2012-2017_OzFlux_{message}.nc',       
                            f'AU-How_2003-2017_OzFlux_{message}.nc',       
                            f'AU-Lit_2016-2017_OzFlux_{message}.nc',       
                            f'AU-Rig_2011-2016_OzFlux_{message}.nc',       
                            f'AU-Rob_2014-2017_OzFlux_{message}.nc',       
                            f'AU-Sam_2011-2017_OzFlux_{message}.nc',       
                            f'AU-Stp_2010-2017_OzFlux_{message}.nc',       
                            f'AU-TTE_2013-2017_OzFlux_{message}.nc',       
                            f'AU-Tum_2002-2017_OzFlux_{message}.nc',       
                            f'AU-Whr_2015-2016_OzFlux_{message}.nc',       
                            f'AU-Wrr_2016-2017_OzFlux_{message}.nc',       
                            f'AU-Ync_2011-2017_OzFlux_{message}.nc',
                            # f'AU-Otw_2009-2010_OzFlux_{message}.nc',     
                            # f'AU-Cow_2010-2015_OzFlux_{message}.nc',         
                            # f'AU-Ctr_2010-2017_OzFlux_{message}.nc',         
                            ]

    site_characters    = pd.read_csv('/g/data/w97/mm3972/scripts/PLUMBER2/LSM_GPP_PLUMBER2/txt/site_character.csv')

    # Correct filtering for Australian sites
    au_site_characters = site_characters[site_characters['site_name'].str.contains('AU')]
    au_site_list       = np.unique(au_site_characters['site_name'])

    for PLUMBER2_AU_file in PLUMBER2_AU_files:

        site_name, lat, lon, df_LAI_P2  = get_PLUMBER2_lat_lon_LAI(PLUMBER2_AU_file, au_site_list, start_date, end_date)
        
        print(lat, lon, df_LAI_P2)
        
        if ~np.isnan(lat):
            # Get pixel LAI in Yuanhua's file
            # Find the indices of the nearest pixels to lat and lon.
            lat_idx_yuan      = np.argmin(np.abs(lat_yuan - lat))
            lon_idx_yuan      = np.argmin(np.abs(lon_yuan - lon))

            # Read the climate_class value of the nearest pixel.
            df_LAI_yuan_tmp   = pd.DataFrame(LAI_yuan[:, lat_idx_yuan, lon_idx_yuan].data, columns=['LAI'])

            # Subset CABLE data to match PLUMBER2 time range
            df_LAI_yuan        = df_LAI_yuan_tmp[(time_yuan >= start_date) & (time_yuan <= end_date)]
            df_LAI_yuan['time']= time_yuan[(time_yuan >= start_date) & (time_yuan <= end_date)]

            # Get pixel LAI in my file
            # Find the indices of the nearest pixels to lat and lon.
            lat_idx_processed    = np.argmin(np.abs(lat_processed - lat))
            lon_idx_processed    = np.argmin(np.abs(lon_processed - lon))

            # Read the climate_class value of the nearest pixel.
            df_LAI_processed_tmp = pd.DataFrame(LAI_processed[:, lat_idx_processed, lon_idx_processed].data, columns=['LAI'])

            # Subset CABLE data to match PLUMBER2 time range
            df_LAI_processed        = df_LAI_processed_tmp[(time_processed >= start_date) & (time_processed <= end_date)]
            df_LAI_processed['time']= time_processed[(time_processed >= start_date) & (time_processed <= end_date)]

            
            # ================== Start Plotting =================
            fig, ax  = plt.subplots(figsize=(6,5))

            # Plot comparison of Qle for CABLE and PLUMBER2
            print(len(df_LAI_yuan['time']), len(df_LAI_processed['time']), len(df_LAI_P2['time']))
            
            ax.plot(df_LAI_yuan['time'],      df_LAI_yuan['LAI'].values, c='red', label='Yuanhua')
            ax.plot(df_LAI_processed['time'], df_LAI_processed['LAI'].values, c='green', label='processed')
            ax.plot(df_LAI_P2['time'],        df_LAI_P2['LAI'].values, c='blue', label='PLUMBER2')
            ax.legend()
            ax.set_title(f'LAI Comparison for {site_name}')

            plt.savefig(f'./Check_LAI_{site_name}_{year}.png',dpi=300)


In [ ]:
lat_yuan_mask = lat_yuan >lats[0] & lat_yuan <lats[1]
lon_yuan_mask = lon_yuan >lons[0] & lon_yuan <lons[1]
LAI_yuan_regmean = np.nanmean(LAI_yuan.loc[:, lat_yuan_mask,lon_yuan_mask],axis=0)

lat_processed_mask    = lat_processed >lats[0] & lat_processed <lats[1]
lon_processed_mask    = lon_processed >lons[0] & lon_processed <lons[1]
LAI_processed_regmean = np.nanmean(LAI_processed.loc[:, lat_processed_mask,lon_processed_mask],axis=0)

### Compare Spatial map

In [ ]:
file_LAI_processed = f'/g/data/w97/mm3972/data/MODIS/MODIS_LAI/AUS/regrid_2_AWAP_5km_daily/remove_high_frequent_varibility_method2_smooth_anomaly/MCD15A3H.061_500m_aid0001_LAI_regridded_daily_2000-2023_remove_high_freq.nc'

f_LAI_processed    = nc.Dataset(file_LAI_processed)
LAI_processed      = f_LAI_processed.variables['LAI'][:]
lat_processed      = f_LAI_processed.variables['latitude'][:]
lon_processed      = f_LAI_processed.variables['longitude'][:]
time_processed     = nc.num2date(f_LAI_processed.variables['time'][:],f_LAI_processed.variables['time'].units,
                     only_use_cftime_datetimes=False, only_use_python_datetimes=True)

nlat               = len(lat_processed)
nlon               = len(lon_processed)

In [ ]:
for year in np.arange(2000,2020):
    
    file_LAI_yuan      = f'/g/data/w97/mm3972/data/MODIS/MODIS_LAI/Sun_Yatsen_Uni_processed_MODIS_LAI_61/lai_8-day_0.05_{year}.nc'

    f_LAI_yuan         = nc.Dataset(file_LAI_yuan)
    LAI_yuan           = f_LAI_yuan.variables['lai'][:]
    lat_yuan           = f_LAI_yuan.variables['lat'][:]
    lon_yuan           = f_LAI_yuan.variables['lon'][:]
    LAI_yuan_mean      = np.mean(LAI_yuan, axis=0)

    # Make time array for processed LAI  
    start_date         = datetime(year, 1, 1, 0, 0)
    end_date           = datetime(year+1, 1, 1, 0, 0)
    LAI_processed_mean = np.mean(LAI_processed[(time_processed>=start_date) & (time_processed<=end_date), :,:], axis=0)

    # Regrid
    LAI_yuan_mean_regrid= regrid_data(lat_yuan, lon_yuan, lat_processed, lon_processed, 
                                      LAI_yuan_mean, method='nearest',threshold=None)

    # ================== Start Plotting =================
    # Create a figure with 3 subplots in 1 row
    fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(18, 5), subplot_kw={'projection': ccrs.PlateCarree()})

    # Adjust spacing between subplots
    plt.subplots_adjust(wspace=0.3)

    # Loop through each axis
    for ax in axs:

        # Set extent based on loc_lat and loc_lon
        ax.set_extent([112, 154, -43, -10])  # Example extent, adjust as needed
        ax.coastlines(resolution="50m", linewidth=1)

    # Plot windspeed based on variable names
    levels1 = [0,0.5,1,1.5,2.,2.5,3,3.5,4.,4.5,5.,5.5,6.,6.5,7.]
    levels2 = [-1.,-0.8,-0.6,-0.4,-0.2,0.2,0.4,0.6,0.8,1.]
    plot1   = axs[0].contourf(lon_processed, lat_processed, LAI_yuan_mean_regrid, levels=levels1, transform=ccrs.PlateCarree(), extend='both', cmap=plt.cm.BrBG)
    plot2   = axs[1].contourf(lon_processed, lat_processed, LAI_processed_mean, levels=levels1, transform=ccrs.PlateCarree(), extend='both', cmap=plt.cm.BrBG)
    plot3   = axs[2].contourf(lon_processed, lat_processed, LAI_processed_mean-LAI_yuan_mean_regrid, levels=levels2, transform=ccrs.PlateCarree(), extend='both', cmap=plt.cm.BrBG)

    axs[0].set_title('Yuanhua', size=16)
    axs[1].set_title('Processed', size=16)
    axs[2].set_title('Processed-Yuanhua', size=16)

    cb = plt.colorbar(plot1, ax=axs[0], orientation="horizontal", pad=0.02, aspect=16, shrink=0.8)
    cb = plt.colorbar(plot2, ax=axs[1], orientation="horizontal", pad=0.02, aspect=16, shrink=0.8)
    cb = plt.colorbar(plot3, ax=axs[2], orientation="horizontal", pad=0.02, aspect=16, shrink=0.8)
    cb.ax.tick_params(labelsize=10)

    # Adjust layout
    plt.tight_layout()
    plt.savefig(f'./plots/Check_LAI/Check_Yuanhua_my_LAI_against_PLUMBER2_{year}.png',dpi=300)

### Check the correlation between LAI and albedo

In [ ]:
from scipy.stats import pearsonr

year               = 2017
file_gridinfo_SG   = f'/g/data/w97/mm3972/scripts/Land_Drought_Rainfall/process_for_CABLE/nc_files/gridinfo_AWAP_OpenLandMap_ELEV_DLCM_fix_MODIS_LAI_albedo_lc_time_varying_{year}.nc'
file_gridinfo      = f'/g/data/w97/mm3972/scripts/Land_Drought_Rainfall/process_for_CABLE/nc_files/no_SG_31-day_running_smooth/gridinfo_AWAP_OpenLandMap_ELEV_DLCM_fix_MODIS_LAI_albedo_lc_time_varying_{year}.nc'

f_SG               = nc.Dataset(file_gridinfo_SG)
LAI_SG             = f_SG.variables['LAI'][:]
albedo             = f_SG.variables['Albedo_MODIS'][:]
lat_lai            = f_SG.variables['latitude'][:]
lon_lai            = f_SG.variables['longitude'][:]

f                  = nc.Dataset(file_gridinfo)
LAI                = f.variables['LAI'][:]

# Make time array for processed LAI  
start_date         = datetime(year, 1, 1, 0, 0)
end_date           = datetime(year, 12, 31, 0, 0)
time               = np.array([start_date + timedelta(days=i) for i in range((end_date - start_date).days + 1)])
    
message            = 'Met'
PLUMBER2_path      = '/g/data/w97/mm3972/data/Fluxnet_data/Post-processed_PLUMBER2_outputs/Nc_files/Met'
PLUMBER2_AU_files  = [  f'AU-ASM_2011-2017_OzFlux_{message}.nc',      
                        f'AU-Cpr_2011-2017_OzFlux_{message}.nc',     
                        f'AU-Cum_2013-2018_OzFlux_{message}.nc',       
                        f'AU-DaP_2009-2012_OzFlux_{message}.nc',       
                        f'AU-DaS_2010-2017_OzFlux_{message}.nc',       
                        f'AU-Dry_2011-2015_OzFlux_{message}.nc',       
                        f'AU-Emr_2012-2013_OzFlux_{message}.nc',       
                        f'AU-GWW_2013-2017_OzFlux_{message}.nc',       
                        f'AU-Gin_2012-2017_OzFlux_{message}.nc',       
                        f'AU-How_2003-2017_OzFlux_{message}.nc',       
                        f'AU-Lit_2016-2017_OzFlux_{message}.nc',       
                        f'AU-Rig_2011-2016_OzFlux_{message}.nc',       
                        f'AU-Rob_2014-2017_OzFlux_{message}.nc',       
                        f'AU-Sam_2011-2017_OzFlux_{message}.nc',       
                        f'AU-Stp_2010-2017_OzFlux_{message}.nc',       
                        f'AU-TTE_2013-2017_OzFlux_{message}.nc',       
                        f'AU-Tum_2002-2017_OzFlux_{message}.nc',       
                        f'AU-Whr_2015-2016_OzFlux_{message}.nc',       
                        f'AU-Wrr_2016-2017_OzFlux_{message}.nc',       
                        f'AU-Ync_2011-2017_OzFlux_{message}.nc',
                        # f'AU-Otw_2009-2010_OzFlux_{message}.nc',     
                        # f'AU-Cow_2010-2015_OzFlux_{message}.nc',         
                        # f'AU-Ctr_2010-2017_OzFlux_{message}.nc',         
                        ]

site_characters    = pd.read_csv('/g/data/w97/mm3972/scripts/PLUMBER2/LSM_GPP_PLUMBER2/txt/site_character.csv')

# Correct filtering for Australian sites
au_site_characters = site_characters[site_characters['site_name'].str.contains('AU')]
au_site_list       = np.unique(au_site_characters['site_name'])

for PLUMBER2_AU_file in PLUMBER2_AU_files:

    site_name, lat, lon, df_LAI_P2  = get_PLUMBER2_lat_lon_LAI(PLUMBER2_AU_file, au_site_list, start_date, end_date)
    
    if ~np.isnan(lat):
        # Find the indices of the nearest pixels to lat and lon.
        lat_idx      = np.argmin(np.abs(lat_lai - lat))
        lon_idx      = np.argmin(np.abs(lon_lai - lon))

        # Read the climate_class value of the nearest pixel.
        df_LAI       = pd.DataFrame(LAI[:, lat_idx, lon_idx].data, columns=['LAI'])

        # Read the climate_class value of the nearest pixel.
        df_LAI_SG    = pd.DataFrame(LAI_SG[:, lat_idx, lon_idx].data, columns=['LAI'])

        # ================== Start Plotting =================
        fig, ax  = plt.subplots(figsize=(6,5))

        ax.plot(time, df_LAI['LAI'].values, c='red', label='LAI')
        ax.plot(time, df_LAI_SG['LAI'].values, c='green', label='LAI_SG')
        ax.plot(time, df_LAI_P2['LAI'].values, c='blue', label='LAI_P2')
        ax.legend()
        ax.set_title(f'LAI Comparison for {site_name}')

        plt.savefig(f'./Check_LAI_gridinfo_{site_name}_{year}.png',dpi=300)
        
        df_albedo            = pd.DataFrame(albedo[0, :, lat_idx, lon_idx].data, columns=['albedo1'])
        df_albedo['albedo2'] = albedo[1, :, lat_idx, lon_idx].data
        df_albedo['albedo3'] = albedo[2, :,  lat_idx, lon_idx].data
        df_albedo['albedo4'] = albedo[3, :,  lat_idx, lon_idx].data
        
        # Pearson correlation
        correlation_LAI_a1, p_value_LAI_a1       = pearsonr(df_LAI['LAI'], df_albedo['albedo1'])
        correlation_LAI_a2, p_value_LAI_a2       = pearsonr(df_LAI['LAI'], df_albedo['albedo2'])
        correlation_LAI_a3, p_value_LAI_a3       = pearsonr(df_LAI['LAI'], df_albedo['albedo3'])
        correlation_LAI_a4, p_value_LAI_a4       = pearsonr(df_LAI['LAI'], df_albedo['albedo4'])
        
        correlation_LAI_SG_a1, p_value_LAI_SG_a1 = pearsonr(df_LAI_SG['LAI'], df_albedo['albedo1'])
        correlation_LAI_SG_a2, p_value_LAI_SG_a2 = pearsonr(df_LAI_SG['LAI'], df_albedo['albedo2'])
        correlation_LAI_SG_a3, p_value_LAI_SG_a3 = pearsonr(df_LAI_SG['LAI'], df_albedo['albedo3'])
        correlation_LAI_SG_a4, p_value_LAI_SG_a4 = pearsonr(df_LAI_SG['LAI'], df_albedo['albedo4'])
        
        correlation_LAI_P2_a1, p_value_LAI_P2_a1 = pearsonr(df_LAI_P2['LAI'], df_albedo['albedo1'])
        correlation_LAI_P2_a2, p_value_LAI_P2_a2 = pearsonr(df_LAI_P2['LAI'], df_albedo['albedo2'])
        correlation_LAI_P2_a3, p_value_LAI_P2_a3 = pearsonr(df_LAI_P2['LAI'], df_albedo['albedo3'])
        correlation_LAI_P2_a4, p_value_LAI_P2_a4 = pearsonr(df_LAI_P2['LAI'], df_albedo['albedo4'])
        print(f'======= {site_name} =======')
        print(correlation_LAI_a1, correlation_LAI_a2, correlation_LAI_a3, correlation_LAI_a4)
        print(correlation_LAI_SG_a1, correlation_LAI_SG_a2, correlation_LAI_SG_a3, correlation_LAI_SG_a4)
        print(correlation_LAI_P2_a1, correlation_LAI_P2_a2, correlation_LAI_P2_a3, correlation_LAI_P2_a4)

### Calculate correl spatial map

In [20]:
# Function to compute correlation map
def compute_correlation_map(data1, data2):
    """
    Calculate the correlation map between two 3D arrays over time.
    
    Args:
        data1: 3D array of shape (time, lat, lon)
        data2: 3D array of shape (time, lat, lon)
    
    Returns:
        2D correlation map (lat, lon)
    """
    assert data1.shape == data2.shape, "Arrays must have the same shape"
    
    correlation_map = np.full((data1.shape[1], data1.shape[2]), np.nan)
    
    for i in range(data1.shape[1]):  # Iterate over latitudes
        for j in range(data1.shape[2]):  # Iterate over longitudes
            series1 = data1[:, i, j]
            series2 = data2[:, i, j]
            
            # Skip grid points with NaN values
            if np.any(np.isnan(series1)) or np.any(np.isnan(series2)):
                correlation_map[i, j] = np.nan
                continue
            
            correlation_map[i, j] = np.corrcoef(series1, series2)[0, 1]
    
    return correlation_map

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import netCDF4 as nc
from scipy.stats import pearsonr
from datetime import datetime, timedelta

# Input year
# year = 2017
for albedo_num in np.arange(1,4):
    for year in np.arange(2000,2021):
        # File paths
        file_gridinfo_SG = f'/g/data/w97/mm3972/scripts/Land_Drought_Rainfall/process_for_CABLE/nc_files/gridinfo_AWAP_OpenLandMap_ELEV_DLCM_fix_MODIS_LAI_albedo_lc_time_varying_{year}.nc'
        file_gridinfo    = f'/g/data/w97/mm3972/scripts/Land_Drought_Rainfall/process_for_CABLE/nc_files/no_SG_31-day_running_smooth/gridinfo_AWAP_OpenLandMap_ELEV_DLCM_fix_MODIS_LAI_albedo_lc_time_varying_{year}.nc'

        # Load data from NetCDF files
        f_SG    = nc.Dataset(file_gridinfo_SG)
        LAI_SG  = f_SG.variables['LAI'][:]  # [time, lat, lon]
        albedo  = f_SG.variables['Albedo_MODIS'][:]  # [time, lat, lon]
        lat_lai = f_SG.variables['latitude'][:]
        lon_lai = f_SG.variables['longitude'][:]

        f       = nc.Dataset(file_gridinfo)
        LAI     = f.variables['LAI'][:]  # [time, lat, lon]

        # Generate time array for processed LAI
        start_date = datetime(year, 1, 1, 0, 0)
        end_date   = datetime(year, 12, 31, 0, 0)
        time       = np.array([start_date + timedelta(days=i) for i in range((end_date - start_date).days + 1)])

        # Compute the correlation map
        correlation_map_SG = compute_correlation_map(LAI_SG, albedo[albedo_num,:,:,:])
        correlation_map    = compute_correlation_map(LAI, albedo[albedo_num,:,:,:])

        # ================== Start Plotting =================
        fig, axs  = plt.subplots(nrows=1, ncols=3, figsize=(6,5), subplot_kw={'projection': ccrs.PlateCarree()})
        levels1   = [-0.95, -0.9,-0.85,-0.8,-0.75,-0.7,-0.65,-0.6,-0.55,-0.5,-0.45,-0.4,-0.35,-0.3,-0.25,-0.2, -0.15,-0.1,-0.05, 0,
                      0.05,  0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8,  0.85, 0.9, 0.95]
        levels2   = [-0.5,-0.45,-0.4,-0.35,-0.3,-0.25,-0.2, -0.15,-0.1,-0.05, 0.05,  0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, ]

        plot1     = axs[0].contourf(lon_lai, lat_lai, correlation_map_SG, levels=levels1, cmap="coolwarm", extend="both")
        cb1       = plt.colorbar(plot1, ax=axs[0], label="LAI_SG vs albedo", orientation="horizontal", pad=0.02, aspect=16, shrink=0.8)
        cb1.ax.tick_params(labelsize=9)

        plot2     = axs[1].contourf(lon_lai, lat_lai, correlation_map, levels=levels1, cmap="coolwarm", extend="both")
        cb2       = plt.colorbar(plot2, ax=axs[1],label="LAI vs albedo",orientation="horizontal", pad=0.02, aspect=16, shrink=0.8)
        cb2.ax.tick_params(labelsize=9)

        plot3     = axs[2].contourf(lon_lai, lat_lai, np.abs(correlation_map_SG)-np.abs(correlation_map), levels=20, cmap="coolwarm", extend="both")
        cb3       = plt.colorbar(plot3, ax=axs[2],label="abs(SG) - abs(noSG)",orientation="horizontal", pad=0.02, aspect=16, shrink=0.8)
        cb3.ax.tick_params(labelsize=9)
        plt.tight_layout()
        plt.savefig(f'/g/data/w97/mm3972/scripts/Land_Drought_Rainfall/plots/Check_LAI/Check_LAI_albedo_correl_{year}_albedo={albedo_num+1}.png',dpi=300)
